<a href="https://colab.research.google.com/github/samilarinc/engineerdocs/blob/main/Dinov2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

!pip install -q timm

#vit_base_patch16_224:    85m parameters (1,768) output
#vit_small_resnet26d_224: 63m parameters (1,768) output
#vit_small_patch16_224:   22m parameters (1,384) output
#vit_tiny_patch16_224:    5m  parameters (1,192) output

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm *.csv
if "csv_folder" not in os.listdir("/content/"):
    os.system('cp drive/MyDrive/csv_folder.zip .')
    os.system('unzip csv_folder.zip -d .')

if "processed_teknofest" not in os.listdir("/content/"):
    os.system('cp /content/drive/MyDrive/processed_teknofest.zip .')
    os.system('unzip processed_teknofest.zip -d .')

task = 0 # int(input('0 for birads, 1 for kompo: '))
batch_size = 16 # int(input('Enter batch size: '))
target_names = [['BI-RADS0', 'BI-RADS1-2','BI-RADS4-5'], ['A', 'B', 'C', 'D']][task]

data_list = ["teknofest"]
if False: #input("Use VinDr (n/y): ") == "y":
  data_list.append("vindr")
  os.system('cp /content/drive/MyDrive/processed_vindr.zip .')
  os.system('unzip processed_vindr.zip -d .')
  os.system('rm processed_vindr/dbca9d28baa3207b3187c4d07dc81a80*')

if input("Use rsna0 (n/y): ") == "y":
  data_list.append("rsna0")
  os.system('cp /content/drive/MyDrive/rsna_birads0_clahe.zip .')
  os.system('unzip rsna_birads0_clahe.zip')
  os.system('mv rsna_birads0_clahe processed_rsna0')
  os.system('cp drive/MyDrive/rsna0_birads_fixed.csv .')

model_name = "vit_tiny_patch16_224" # input("Enter model name (timm or torch): ") #vit_tiny_patch16_224

Use rsna0 (n/y): n


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import gc
import cv2
import time
import timm

from tqdm import tqdm
from time import perf_counter
from torchsummary import summary

import torch
t = torch
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import torchvision as tv
from torchvision import io
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchsummary import summary

import pandas as pd
from PIL import Image
import warnings
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore", category=UserWarning)

device = 'cuda' if t.cuda.is_available() else 'cpu'
# !nvidia-smi

In [ ]:
class MemeDataset(Dataset):
    def __init__(self, image_root_dir, transform, csv_file, model_type = 'MEME KOMPOZİSYONU'):
        self.data = pd.read_csv(csv_file, index_col = 0)
        y = pd.get_dummies(self.data[model_type], prefix = '')
        if model_type != 'MEME KOMPOZİSYONU':
            if image_root_dir.split('_')[1] == 'rsna':
                self.data['one_hot'] = list(np.hstack((np.array(y), np.zeros((y.shape[0], 1)))))

            if image_root_dir.split('_')[1] == 'rsna0':
                self.data['one_hot'] = list(np.hstack((np.array(y), np.zeros((y.shape[0], 1)))))

            elif image_root_dir.split('_')[1] == 'vindr':
                self.data['one_hot'] = list(np.hstack((np.zeros((y.shape[0], 1)), np.array(y))))

            else:
                self.data['one_hot'] = list(np.array(y))
        else:
            self.data['one_hot'] = list(np.array(y))
        
        self.image_root_dir = image_root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if t.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.image_root_dir,
                                str(self.data.iloc[idx, 0]))

        LCC =  io.read_image(img_path + '_LCC.png')
        LMLO = io.read_image(img_path + '_LMLO.png')
        RCC =  io.read_image(img_path + '_RCC.png')
        RMLO = io.read_image(img_path + '_RMLO.png')

        if LCC.shape[0] != 3:
          LCC = t.cat([LCC, LCC, LCC], dim=0)
          RCC = t.cat([RCC, RCC, RCC], dim=0)
          LMLO = t.cat([LMLO, LMLO, LMLO], dim=0)
          RMLO = t.cat([RMLO, RMLO, RMLO], dim=0)

        if self.transform:
            LCC = self.transform(LCC)
            RCC = self.transform(RCC)
            LMLO = self.transform(LMLO)
            RMLO = self.transform(RMLO)

        label = self.data.iloc[idx, 1]

        sample = {'LCC': LCC, 'LMLO': LMLO, 'RCC': RCC, 'RMLO': RMLO,
                  'label': label}
                  
        return sample

class CustomDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
        

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if t.is_tensor(idx):
            idx = idx.tolist()

        image_path = self.df.iloc[idx, -1]
        LCC =  io.read_image(image_path + '_LCC.png')
        LMLO = io.read_image(image_path + '_LMLO.png')
        RCC =  io.read_image(image_path + '_RCC.png')
        RMLO = io.read_image(image_path + '_RMLO.png')

        if LCC.shape[0] != 3:
          LCC = t.cat([LCC, LCC, LCC], dim=0)
          RCC = t.cat([RCC, RCC, RCC], dim=0)
          LMLO = t.cat([LMLO, LMLO, LMLO], dim=0)
          RMLO = t.cat([RMLO, RMLO, RMLO], dim=0)

        label = self.df.iloc[idx, 2]

        if self.transform:
            LCC = self.transform(LCC)
            RCC = self.transform(RCC)
            LMLO = self.transform(LMLO)
            RMLO = self.transform(RMLO)
      

        sample = {'LCC': LCC, 'LMLO': LMLO, 'RCC': RCC, 'RMLO': RMLO, 'label': label}
        return sample

In [ ]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, reduction=self.reduction, weight=self.weight)
        pt = t.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [ ]:
import torch

class AttentionFuser(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionFuser, self).__init__()
        
        self.hidden_size = hidden_size
        
        # Define the linear layers for query, key, and value vectors
        self.query_linear = nn.Linear(384, hidden_size)
        self.key_linear = nn.Linear(384, hidden_size)
        self.value_linear = nn.Linear(384, hidden_size)
        
    def forward(self, extractor_outputs):
        # Compute the attention weights for each vector
        attn_weights = []
        for x in extractor_outputs:
            # Apply the linear layers to reduce the dimensionality of the input vector
            query = self.query_linear(x)
            key = self.key_linear(x)
            value = self.value_linear(x)

            # Compute the attention scores
            attn_scores = torch.bmm(query, key.transpose(1, 2))
            attn_weights.append(F.softmax(attn_scores, dim=2))

        # Combine the attention weights with the corresponding vectors
        fused_output = torch.zeros((extractor_outputs[0].shape[0], 1, self.hidden_size), device=extractor_outputs[0].device)
        for i, x in enumerate(extractor_outputs):
            weighted_x = torch.bmm(attn_weights[i], value)
            fused_output += weighted_x

        return fused_output

class GlobalTransformerEncoder(nn.Module):
    def __init__(self, input_size, num_layers, num_heads, hidden_size, dropout_rate):
        super(GlobalTransformerEncoder, self).__init__()
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, num_heads, hidden_size, dropout_rate),
            num_layers
        )

    def forward(self, x):
        x = self.transformer_encoder(x)
        return x

class DortBasliMemeNet(nn.Module):
    def __init__(self, f_extractor:str, fc:nn.Sequential, fuse):
        super().__init__()

        self.Extractor = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
        #self.MLO_Extractor = timm.create_model(f_extractor, pretrained=True, num_classes=0)
        self.fuse = fuse
        self.fc = fc

    def forward(self, X):
        LCC, LMLO, RCC, RMLO = X['LCC'].to(device), X['LMLO'].to(device), X['RCC'].to(device), X['RMLO'].to(device)
        fLCC = self.Extractor(LCC)
        fRCC = self.Extractor(RCC)
        fLMLO = self.Extractor(LMLO)
        fRMLO = self.Extractor(RMLO)
      
        # exct = torch.cat()

        fused = t.cat((fLCC, fLMLO, fRCC, fRMLO), dim = -1)
        #fused = [LCC.unsqueeze(1), LMLO.unsqueeze(1), RCC.unsqueeze(1), RMLO.unsqueeze(1)]
        # print(fused.shape)
        #fused = self.fuse(fused)
        # print(fused.shape)
        return fused

transform_0 = T.Compose([
    T.AutoAugment(T.AutoAugmentPolicy.IMAGENET),
    T.ConvertImageDtype(t.float)
])
#  = T.Compose([
#     T.GaussianBlur((3, 5)),
#     T.RandomAffine((-30, 30), translate=(0.1, 0.3), scale=(0.5, 0.75)),
#     T.RandomInvert(p = .2),
#     T.ElasticTransform(),
#     T.RandomErasing(p = .25),
#     T.ColorJitter(brightness=0.1, contrast=0.1),
#     T.ConvertImageDtype(t.float),
# ])

transform = transform_0#T.Compose([
#     T.RandomRotation((-30, 30)),
#     T.GaussianBlur((5, 9)),
#     T.ConvertImageDtype(t.float)
# ])

transform_test = T.Compose([
    T.ConvertImageDtype(t.float)    
])

tasks = ('birads', 'kompozisyon')
types = ('BIRADS KATEGORİSİ', 'MEME KOMPOZİSYONU')

train_sets, val_sets = [], []

for i in data_list:
    dataset = MemeDataset(image_root_dir=f'processed_{i}', transform=transform, csv_file = f'{i}_{tasks[task]}_fixed.csv', model_type = types[task])

    # convert data to DataFrame
    df = dataset.data
    df["image_path"] = df["HASTANO"].apply(lambda x: os.path.join(dataset.image_root_dir, str(x)))

    # split data in a stratified way
    if i == 'teknofest':
        train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[types[task]])
        val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42, stratify=test_val_df[types[task]])
        test_set = CustomDataset(test_df, transform=transform_test)
    
    else:
      if i == "vindr":
          df = df[df["BIRADS KATEGORİSİ"] != "BI-RADS1-2"]
      train_df, val_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df[types[task]])
        
    
    train_sets.append(CustomDataset(train_df, transform=transform))
    val_sets.append(CustomDataset(val_df, transform=transform_test))

train_set = ConcatDataset(train_sets)
val_set = ConcatDataset(val_sets)

train_loader = DataLoader(train_set, batch_size, True)
val_loader = DataLoader(val_set, batch_size, False)
test_loader = DataLoader(test_set, batch_size, False)

num_class = train_sets[0].df['one_hot'].iloc[0].shape[0]
counts = np.zeros(num_class)
for x in train_sets:
    bi_mat = x.df["one_hot"].to_numpy()
    for i in bi_mat:
        ind = np.argmax(i)
        counts[ind] += 1

print(counts)

a = np.array(counts)
b = np.sum(a) / (num_class * a)
class_weights = t.tensor(b, dtype=t.float)
class_weights

[1527. 3101. 2886.]


tensor([1.6403, 0.8077, 0.8679])

In [ ]:
# fuse = DepthwiseConv(6144, 512)
# fuse = nn.Identity()
fuse = nn.Identity()
#fuse = AttentionFuser(224)

fc_birads = fuse
model = DortBasliMemeNet(model_name, fc_birads, fuse).to(device)

tot_param = 0
for params in model.parameters():
  tot_param += params.numel()
print(f"The model has {tot_param:,} parameters.")

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:01<00:00, 208MB/s]


The model has 86,580,480 parameters.


In [ ]:
total = len(train_loader)

test_total = len(test_loader)
# we will generate 10 times more training data than the original training data size using data augmentation
train_features = []
train_labels = []
val_features = []
val_labels = []
test_features = []
test_labels = []

model.eval()
index  = 0 
for e in range(1):
    for sample in tqdm(train_loader):
        label = sample['label'].to(device)
        out =  model(sample).detach().cpu().numpy()
        # each generated train feature will be a 512 element vector
        train_features.append(out)
        # labels is in GPU, so transfer it to CPU first to assign to the numpy array
        train_labels.append(label.cpu())
        index += len(sample)

index = 0
for sample in tqdm(val_loader):
    labels = sample['label'].to(device)
    val_features.append(model(sample).detach().cpu().numpy())
    val_labels.append(labels.cpu())
    index += len(sample)

index = 0
for sample in tqdm(test_loader):
    labels = sample['label'].to(device)
    test_features.append(model(sample).detach().cpu().numpy())
    test_labels.append(labels.cpu())
    index += len(sample)

train_features = np.concatenate(train_features)
train_labels = np.concatenate(train_labels)
val_features = np.concatenate(val_features)
val_labels = np.concatenate(val_labels)
test_features = np.concatenate(test_features)
test_labels = np.concatenate(test_labels)

100%|██████████| 101/101 [01:42<00:00,  1.02s/it]


In [ ]:
import lightgbm as lgb
import numpy as np
import torch
from sklearn.metrics import accuracy_score

num_feat = out.shape[-1]
# Define the data and labels
train_features_flatten = train_features.reshape(-1, num_feat)
train_labels_flatten = np.argmax(train_labels, axis = 1)
val_features_flatten = val_features.reshape(-1, num_feat)
val_labels_flatten = np.argmax(val_labels, axis = 1)
test_features_flatten = test_features.reshape(-1, num_feat)
test_labels_flatten = np.argmax(test_labels, axis = 1)

lgb_data_train = lgb.Dataset(train_features_flatten, label=train_labels_flatten)
lgb_data_val = lgb.Dataset(val_features_flatten, label=val_labels_flatten)
lgb_data_test = lgb.Dataset(test_features_flatten, label=test_labels_flatten)

# Set up hyperparameters for the LightGBM model
params = {
    "objective": "multiclass",
    "num_class": 3,
    "metric": "multi_logloss",
    "boosting_type": "gbdt",
    "learning_rate": 0.01,
    "num_leaves": 21,
    "max_depth": -1,
    "min_child_samples": 30,
    "subsample": 1.0,
    "subsample_freq": 0,
    "colsample_bytree": 0.8, # 0-1 arası dişi 0.6 falan yap
    "reg_alpha": 0.1,
    "reg_lambda": 0.1,
    "n_jobs": -1,
    "verbose": 1
}

# Train the LightGBM model
num_rounds = 1000
model = lgb.train(params, lgb_data_train, num_rounds, valid_sets = [lgb_data_val], early_stopping_rounds=10, verbose_eval=True)

# Evaluate the model on the training data
predictions = model.predict(train_features_flatten)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = np.sum(predicted_labels == train_labels_flatten) / len(train_labels_flatten)
print(f"Train Accuracy: {accuracy}")

predictions = model.predict(val_features_flatten)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = np.sum(predicted_labels == val_labels_flatten) / len(val_labels_flatten)
print(f"Validation Accuracy: {accuracy}")

predictions = model.predict(test_features_flatten)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = np.sum(predicted_labels == test_labels_flatten) / len(test_labels_flatten)
print(f"Test Accuracy: {accuracy}")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.812351 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 783360
[LightGBM] [Info] Number of data points in the train set: 7514, number of used features: 3072
[LightGBM] [Info] Start training from score -1.593463
[LightGBM] [Info] Start training from score -0.885043
[LightGBM] [Info] Start training from score -0.956896
[1]	valid_0's multi_logloss: 1.0556
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 1.05468
[3]	valid_0's multi_logloss: 1.05384
[4]	valid_0's multi_logloss: 1.05295
[5]	valid_0's multi_logloss: 1.05205
[6]	valid_0's multi_logloss: 1.05118
[7]	valid_0's multi_logloss: 1.05041
[8]	valid_0's multi_logloss: 1.04952
[9]	valid_0's multi_logloss: 1.04866
[10]	valid_0's multi_logloss: 1.04779
[11]	valid_0's multi_logloss: 1.04703
[12]	valid_0's multi_logloss: 1.04623
[13]	valid_0's multi_logloss: 1.0452